# Import delle librerie necessarie

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# !git clone https://github.com/AntonioDelleCanne/tesi.git

In [3]:
%cd DeepLearning_Financial

/data/home/dsvm_server_admin/notebooks/fastai/tesi/DeepLearning_Financial


In [4]:
## EXTERNAL
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import yfinance
from pandas import Series
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit, PredefinedSplit
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from IPython.display import display
import datetime
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
import time
import os
import random 
from sklearn.datasets import make_regression
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetRegressor
from torch.nn.modules.loss import MSELoss
import tensorflow as tf
from tensorflow import keras
from skorch.dataset import CVSplit
from skorch import callbacks
import pickle
from sklearn.model_selection import train_test_split
from functools import partial
import skorch
import pywt
from sklearn import preprocessing
import joblib


##INTERNAL
from models import Autoencoder, Sequence, waveletSmooth, SequenceDouble, SequenceDoubleAtt, SequenceAtt
from utils import prepare_data_lstm, ExampleDataset, save_checkpoint, evaluate_lstm, backtest

In [5]:
def save(model, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(model, f)
        
def load(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [6]:
#returns open high low close volume
def get_index(index="^DJI", start_date="2000-01-01", end_date="2018-12-31"):
    security = yfinance.Ticker(index)# TODO trova mercato asiatico e indiano
    security_data = security.history(start=start_date, end=end_date, actions=False)
    return security_data

In [7]:
def split_index(security_data):
    return security_data["Open"], security_data["High"], security_data["Low"], security_data["Close"], security_data["Volume"]

# Allenamento modello

## Metriche
Come metrica principale per valutare la bonta' dele predizioni utilizzeremo il Return of Investment (ROI).

Con questa metrica assumiamo di utilizzare l'algoritmo di trading precedentemente descritto, e calcoliamo il guadango che avremmo ottenuto se lo avessimo utilizzato sul dataset che stiamo valutando.

In [8]:
def gain(C, C_pred, opn):
    O = opn.reindex_like(C)
    CO_diff = C - O
    growth = C_pred > O
    decline = C_pred < O
    return CO_diff[growth].sum() - CO_diff[decline].sum()
def roi(C, C_pred, opn):
    mean_opn = opn.reindex_like(C).mean()
    return gain(C, C_pred, opn) / mean_opn

## Preparazione dei dati
Come spiegato in precedenza, visto che alcune feature non sono disponibili all'inizio della giornata, per poterle utilizzare nel nostro modello, utilizzeremo i dati dei giori passati, servendoci della funzione shift.

Visto che l'utilizzo di questa funzione fara' si che in alcune riche vi saranno dei vaolri NaN, dobbiamo assicurarci di eliminare queste osservazioni sia nelle serie relative ale features che in quella della variabile da predire.
Questo compito e' svolto dalla funzione prepare_data.

In [ ]:
def prepare_data(features, target):
    X = pd.DataFrame(features)
    X.dropna(inplace=True)
    Y = target.reindex_like(X)
    return X, Y

Con questa funzione dividiamo i dati in training set e validation set come è stato fatto nel paper

In [ ]:
def s_split_before_2010_06_30(X):
    is_train = X.index.date < datetime.date(2010,6,30)
    X_train = X.loc[is_train]
    X_val = X.loc[~is_train]
    return X_train, X_val

In [ ]:
def split_before_2010_06_30(X, y):
    is_train = X.index.date < datetime.date(2010,6,30)
    X_train = X.loc[is_train]
    y_train = y.loc[is_train]
    X_val = X.loc[~is_train]
    y_val = y.loc[~is_train]
    return X_train, X_val, y_train, y_val

# Preparazione del dataset

In [12]:
# def get_dataset(opn, close, high, low, volume, name, usd_index='DX-Y.NYB', train=size, encoder='ohlcv_encoder_DJIA', encoder_stats='ohlcv_stats_encoder_DJIA'):
#     features={}
#     dataset = None
#     volume = volume/1000 # prescaling the volume
    
#     us_dollar_opn = get_index(usd_index, start_date=opn.index.min(), end_date=opn.index.max())["Open"]
    
#     if(zero_mean):
#         opn = zero_mean(opn)
#         close = zero_mean(close)
#         high = zero_mean(high)
#         low = zero_mean(low)
#         volume = zero_mean(volume)
#         us_dollar_opn = zero_mean(us_dollar_opn)
    
#     if(wavelet):
#         opn = apply_wavelet_transform(opn)
#         close = apply_wavelet_transform(close)
#         high = apply_wavelet_transform(high)
#         low = apply_wavelet_transform(low)
#         volume = apply_wavelet_transform(volume)
#         us_dollar_opn = apply_wavelet_transform(us_dollar_opn)
    
#     #calucate derived indicators
#     TP = ((high + low + close) / 3 ).shift(1)
#     trs = pd.DataFrame(index=high.index)
#     trs['tr0'] = abs(high - close)
#     trs['tr1'] = abs(high - close.shift(1))
#     trs['tr2'] = abs(low - close.shift(1))
#     TR = trs[['tr0', 'tr1', 'tr2']].max(axis=1).shift(1)
#     ema20 = opn.ewm(span=20).mean()
#     ma10 = opn.rolling(window=10).mean()
#     ma5 = opn.rolling(window=5).mean()
#     macd = opn.ewm(span=26).mean() - opn.ewm(span=12).mean()
#     cci_ndays=20
#     cci = (TP - TP.rolling(cci_ndays).mean()) / (0.015 * TP.rolling(cci_ndays).std())
#     atr = TR.ewm(span = 10).mean()
#     ma20 = opn.rolling(window=20).mean()
#     std20 = opn.rolling(window=20).std()
#     k=2
#     boll_up =  ma20 + (k*std20)
#     boll_down = ma20 - (k*std20)
#     roc = (opn - opn.shift(9))*100/opn.shift(9)
#     mtm6 = (opn - opn.shift(127))
#     mtm12 = (opn - opn.shift(253)) #length of a trading year is on average 253 days
#     wvad = (((close - low) - (high - close)) * volume/(high - low)).shift(1)
#     smi = (close - (high - low)/2).shift(1)
    
    
#     ohlcv_encoder = load(encoder)
# #     ohlcv_stats_encoder = load(encoder_stats)
#     if(z_norm):
#         opn= z_score(opn)
#         close = z_score(close)
#         high = z_score(high)
#         low = z_score(low)
#         volume = z_score(volume)
#     target=close
#     if(name is "open"):
#         features["Open"] = opn
#     elif(name is "ohlcv" or "sa_ohlcv"):
#         features["Open"] = opn
#         features["CloseL1"] = close.shift(1)
#         features["HighL1"] = high.shift(1)
#         features["LowL1"] = low.shift(1)
#         features["VolumeL1"] = volume.shift(1)
#         if(name is "sa_ohlcv"):
#             #ricava altri parametri e aggiungi a features
#             temp_dset = prepare_data(features, target)
#             features = {}
            
#             for i, feature in enumerate(encode(temp_dset[0].to_numpy().astype(np.float32), ohlcv_encoder).T):
#                 features["deep{}".format(i)] = pd.Series(feature, index=temp_dset[0].index)
#     elif(name is "ohlcv_stats" or "sa_ohlcv_stats"):
#         features["Open"] = opn
#         features["CloseL1"] = close.shift(1)
#         features["HighL1"] = high.shift(1)
#         features["LowL1"] = low.shift(1)
#         features["VolumeL1"] = volume.shift(1)
#         features["EMA20"] = ema20
#         features["MA5"] = ma5
#         features["MA10"] = ma10
#         features["MA20"] = ma20
#         features["MACD"] = macd
#         features["CCI"] = cci
#         features["ATR"] = atr
#         features["BollUp"] = boll_up
#         features["BollDown"] = boll_down
#         features["WVAD"] = wvad
#         features["MTM6"] = mtm6
#         features["MTM12"] = mtm12
#         features["SMI"] = smi
#         features["ROC"] = roc
#         features["USD_Open"] = us_dollar_opn 
#         #TODO
#         if(name is "sa_ohlcv_stats"):
#             #ricava altri parametri e aggiungi a features
#             temp_dset = prepare_data(features, target)
#             features = {}
#             for i, feature in enumerate(encode(temp_dset[0].to_numpy().astype(np.float32), ohlcv_encoder).T):
#                 features["deep{}".format(i)] = pd.Series(feature, index=temp_dset[0].index)
#     else:
#         raise Exception('Nome del feature-set non valido')
        
#     return prepare_data(features, target)

In [75]:
def get_ext_feats(ohlcv):
    res = ohlcv.copy()
    
    opn = res["Open"]
    close = res["Close"]
    high = res["High"]
    low = res["Low"]
    volume = res["Volume"]
    
    #calucate derived indicators
    TP = ((high + low + close) / 3 ).shift(1)
    trs = pd.DataFrame(index=high.index)
    trs['tr0'] = abs(high - close)
    trs['tr1'] = abs(high - close.shift(1))
    trs['tr2'] = abs(low - close.shift(1))
    TR = trs[['tr0', 'tr1', 'tr2']].max(axis=1).shift(1)
    ema20 = opn.ewm(span=20).mean()
    ma10 = opn.rolling(window=10).mean()
    ma5 = opn.rolling(window=5).mean()
    macd = opn.ewm(span=26).mean() - opn.ewm(span=12).mean()
    cci_ndays=20
    cci = (TP - TP.rolling(cci_ndays).mean()) / (0.015 * TP.rolling(cci_ndays).std())
    atr = TR.ewm(span = 10).mean()
    ma20 = opn.rolling(window=20).mean()
    std20 = opn.rolling(window=20).std()
    k=2
    boll_up =  ma20 + (k*std20)
    boll_down = ma20 - (k*std20)
    roc = (opn - opn.shift(9))/opn.shift(9)
    mtm6 = (opn - opn.shift(127))
    mtm12 = (opn - opn.shift(253)) #length of a trading year is on average 253 days
    wvad = (((close - low) - (high - close)) * volume/(high - low)).shift(1)
    smi = (close - (high - low)/2).shift(1)
    
    res["CloseL1"] = close.shift(1)
    res["HighL1"] = high.shift(1)
    res["LowL1"] = low.shift(1)
    res["VolumeL1"] = volume.shift(1)
    res["EMA20"] = ema20
    res["MA5"] = ma5
    res["MA10"] = ma10
    res["MA20"] = ma20
    res["MACD"] = macd
    res["CCI"] = cci
    res["ATR"] = atr
    res["BollUp"] = boll_up
    res["BollDown"] = boll_down
    res["WVAD"] = wvad
    res["MTM6"] = mtm6
    res["MTM12"] = mtm12
    res["SMI"] = smi
    res["ROC"] = roc
    
    return res

In [79]:
# takes as input ohlcv dataframe
def get_data_set(ohlcv, ext_feats=True, usd_index='DX-Y.NYB', wavelet=True):
    feats = ohlcv.copy()
    usd_open = get_index(usd_index, start_date=opn.index.min(), end_date=opn.index.max())["Open"]
    
    if(wavelet):
    #apply transforms
        for f_name in ('Open', 'Close', 'High', 'Low'):
            feats[f_name] = apply_wavelet_transform(feats[f_name]) 
        usd_open = apply_wavelet_transform(usd_open)
    
    if(ext_feats):
        feats = get_ext_feats(feats)
        feats["USDOpen"]  = usd_open
    return feats

In [80]:
def get_dataset_by_name(ohlcv, name):
    if(name is "open"):
        return get_data_set(ohlcv, ext_feats=False)[["Open", "Close"]]
    elif(name is "ohlcv"):
        return get_data_set(ohlcv, ext_feats=False)
    elif(name is "ext"):
        return get_data_set(ohlcv)
    raise Exception('Nome del feature-set non valido')

In [15]:
#TODO per features con SA
# if(name is "sa_ohlcv"):
#             #ricava altri parametri e aggisungi a features
#             temp_dset = prepare_data(features, target)
#             features = {}
            
#             for i, feature in enumerate(encode(temp_dset[0].to_numpy().astype(np.float32), ohlcv_encoder).T):
#                 features["deep{}".format(i)] = pd.Series(feature, index=temp_dset[0].index)

## Regolarizzazione

### Wavelet

In [16]:
def apply_wavelet_transform(data, consider_future=False, wavelet='haar'):
    res = data.copy()
    if(consider_future):
        res, _ = pywt.dwt(data.copy(), wavelet=wavelet)
    else:
        for i in range(res.shape[0]):
            if(i > 0):
                cA =  waveletSmooth(data.iloc[:i+1].copy(), wavelet=wavelet, level=4, DecLvl=3)
                res.iloc[i] = cA[-1]
    return res

In [17]:
# def apply_wavelet_transform(data, consider_future=False, wavelet='haar'):
#     res = data.copy()
#     if(consider_future):
#         res, _ = pywt.dwt(data.copy(), wavelet=wavelet)
#     else:
#         for i in range(res.shape[0]):
#             if(i > 0):
#                 cA, cD = pywt.dwt(data.iloc[:i+1].copy(), wavelet=wavelet)
#                 res.iloc[i] = cA[-1]
#     return res

In [18]:
opn =get_index()["Open"]
haar = apply_wavelet_transform(opn, consider_future=False, wavelet='haar')
coif3 = apply_wavelet_transform(opn, consider_future=False, wavelet='coif3')

/data/anaconda/envs/fastai/lib/python3.6/site-packages/pywt/_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))
/data/anaconda/envs/fastai/lib/python3.6/site-packages/pywt/_thresholding.py:23: RuntimeWarning: invalid value encountered in true_divide
  thresholded = (1 - value/magnitude)


In [19]:
fig, ax = plt.subplots(3,1, sharex=False, figsize=(20,15))
ax[0].plot(opn, label='open')
ax[1].plot(haar, label='haar wavelet')
ax[2].plot(coif3, label ='coif3 wavelet')
ax[0].legend()
ax[1].legend()
ax[2].legend()

## Normalizzazione 
Le seguenti funzioni sono utilizzate per normalizzare i dati, pianifichiamo di utilizzare queste funzioni anche sulla variabile da predire dovremo implementarne anche l'inversa, di modo da poter denormalizzare le predizioni fatte dal modello.
Questo passo sarà importante nella valutazione del modello, ad esempio per calcolare il ROI.

## Scaler

In [20]:
data = get_index()
ohlcv_dataset = get_data_set(data, ext_feats=False).dropna()
ext_dataset = get_data_set(data).dropna()

/data/anaconda/envs/fastai/lib/python3.6/site-packages/pywt/_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))
/data/anaconda/envs/fastai/lib/python3.6/site-packages/pywt/_thresholding.py:23: RuntimeWarning: invalid value encountered in true_divide
  thresholded = (1 - value/magnitude)


In [21]:
# ohlcv_scaler = StandardScaler()
# ext_scaler = StandardScaler()
# ohlcv_scaler.fit(ohlcv_dataset)
# ext_train = ext_scaler.fit_transform(s_split_before_2010_06_30(ext_dataset)[0])
# ohlcv_train = ohlcv_scaler.fit_transform(s_split_before_2010_06_30(ohlcv_dataset)[0])

In [22]:
# #save scalers
# joblib.dump(ext_scaler, 'ext_scaler.bin', compress=True)
# joblib.dump(ohlcv_scaler, 'ohlcv_scaler.bin', compress=True)

In [23]:
ext_scaler=joblib.load('ext_scaler.bin')
ohlcv_scaler = joblib.load('ohlcv_scaler.bin')

In [24]:
#TODO prendi training set per tutte features
# poi allena entrambi scalers(vedi se basta solo quello ext)
# poi allena atoencoder su train scalato

## Stacked Autoencoder

In [36]:
def get_encoder(X, val=None, sa_hidden_size=10):
    X_train_f = X.astype(np.float32)
    if(val is not None):
        X_val_f = val.astype(np.float32)
    #Initialize the autoencoder
    sa_hidden_size= np.ceil(X.shape[1] / 2).astype(int) # Con tutte le features 10

    num_hidden_1 = sa_hidden_size
    num_hidden_2 = sa_hidden_size
    num_hidden_3 = sa_hidden_size
    num_hidden_4 = sa_hidden_size

    n_epoch1=15000 #10000
    n_epoch2 = 2000
    n_epoch3 = 600
    n_epoch4 = 500
    batch_size=20

    # ---- train using training data

    # The n==0 statement is done because we only want to initialize the network once and then keep training
    # as we move through time 

    auto1 = Autoencoder(X_train_f.shape[1], num_hidden_1)
    auto2 = Autoencoder(num_hidden_1, num_hidden_2)
    auto3 = Autoencoder(num_hidden_2, num_hidden_3)
    auto4 = Autoencoder(num_hidden_3, num_hidden_4)
    
    # Train the autoencoder 
    # switch to training mode
    auto1.train()      
    auto2.train()
    auto3.train()
    auto4.train()

    inputs = torch.from_numpy(X_train_f)
    val_in = torch.from_numpy(X_val_f)
    auto1.fit(X_train_f, X_val_f, n_epoch=n_epoch1, batch_size=batch_size)

    auto1_out = auto1.encoder(inputs).data.numpy()
    val1_out = auto1.encoder(val_in).data.numpy()
    auto2.fit(auto1_out, val1_out, n_epoch=n_epoch2, batch_size=batch_size)


    auto1_out = torch.from_numpy(auto1_out.astype(np.float32))
    auto2_out = auto2.encoder(auto1_out).data.numpy()
    val1_out = torch.from_numpy(val1_out.astype(np.float32))
    val2_out = auto2.encoder(val1_out).data.numpy()
    auto3.fit(auto2_out, val2_out, n_epoch=n_epoch3, batch_size=batch_size)


    auto2_out = torch.from_numpy(auto2_out.astype(np.float32))
    auto3_out = auto3.encoder(auto2_out).data.numpy()
    val2_out = torch.from_numpy(val2_out.astype(np.float32))
    val3_out = auto3.encoder(val2_out).data.numpy()
    auto4.fit(auto3_out, val3_out, n_epoch=n_epoch4, batch_size=batch_size)

    # Change to evaluation mode, in this mode the network behaves differently, e.g. dropout is switched off and so on
    auto1.eval()        
    auto2.eval()
    auto3.eval()
    auto4.eval()
    return [auto1, auto2, auto3, auto4]

In [27]:
def encode(feat_matrix, encoder):
    encoded = torch.from_numpy(feat_matrix)
    for auto in encoder:
        encoded = auto.encoder(encoded)
    return encoded.data.numpy()
    

In [28]:
def encoder_score(data, encoder):
    for auto in encoder:
        auto.eval()
    auto
    for auto in encoder:
        auto.train()

In [29]:
ext_train = ext_scaler.transform(s_split_before_2010_06_30(ext_dataset)[0])
ext_eval = ext_scaler.transform(s_split_before_2010_06_30(ext_dataset)[1])
ohlcv_train = ohlcv_scaler.transform(s_split_before_2010_06_30(ohlcv_dataset)[0])

In [37]:
ext_encoder = get_encoder(ext_train, ext_eval, sa_hidden_size=10)

Data Shuffled
Epoch 1/15000 | train loss: 4.3169 | train_error 0.7174 | valid_error 1.4258
Data Shuffled
Epoch 2/15000 | train loss: 3.4086 | train_error 0.4778 | valid_error 1.4825
Data Shuffled
Epoch 3/15000 | train loss: 2.6773 | train_error 0.4221 | valid_error 1.4717
Data Shuffled
Epoch 4/15000 | train loss: 1.9764 | train_error 0.3893 | valid_error 1.4243
Data Shuffled
Epoch 5/15000 | train loss: 1.5407 | train_error 0.3613 | valid_error 1.3773
Data Shuffled
Epoch 6/15000 | train loss: 1.2482 | train_error 0.3361 | valid_error 1.3372
Data Shuffled
Epoch 7/15000 | train loss: 1.1073 | train_error 0.3160 | valid_error 1.3031
Data Shuffled
Epoch 8/15000 | train loss: 1.0156 | train_error 0.3001 | valid_error 1.2739
Data Shuffled
Epoch 9/15000 | train loss: 0.9952 | train_error 0.2882 | valid_error 1.2543
Data Shuffled
Epoch 10/15000 | train loss: 0.9700 | train_error 0.2778 | valid_error 1.2377
Data Shuffled
Epoch 11/15000 | train loss: 0.9569 | train_error 0.2688 | valid_error 1.22

Epoch 90/15000 | train loss: 1.3016 | train_error 0.0574 | valid_error 0.9027
Data Shuffled
Epoch 91/15000 | train loss: 1.2982 | train_error 0.0594 | valid_error 0.9030
Data Shuffled
Epoch 92/15000 | train loss: 1.2967 | train_error 0.0569 | valid_error 0.9023
Data Shuffled
Epoch 93/15000 | train loss: 1.3039 | train_error 0.0557 | valid_error 0.9012
Data Shuffled
Epoch 94/15000 | train loss: 1.3069 | train_error 0.0562 | valid_error 0.9009
Data Shuffled
Epoch 95/15000 | train loss: 1.3020 | train_error 0.0552 | valid_error 0.8993
Data Shuffled
Epoch 96/15000 | train loss: 1.2935 | train_error 0.0559 | valid_error 0.8988
Data Shuffled
Epoch 97/15000 | train loss: 1.2948 | train_error 0.0547 | valid_error 0.8978
Data Shuffled
Epoch 98/15000 | train loss: 1.2891 | train_error 0.0537 | valid_error 0.8954
Data Shuffled
Epoch 99/15000 | train loss: 1.2901 | train_error 0.0534 | valid_error 0.8953
Data Shuffled
Epoch 100/15000 | train loss: 1.2908 | train_error 0.0537 | valid_error 0.8945
D

Epoch 180/15000 | train loss: 1.3148 | train_error 0.0351 | valid_error 0.8405
Data Shuffled
Epoch 181/15000 | train loss: 1.3076 | train_error 0.0359 | valid_error 0.8402
Data Shuffled
Epoch 182/15000 | train loss: 1.3094 | train_error 0.0353 | valid_error 0.8387
Data Shuffled
Epoch 183/15000 | train loss: 1.3186 | train_error 0.0340 | valid_error 0.8380
Data Shuffled
Epoch 184/15000 | train loss: 1.3148 | train_error 0.0346 | valid_error 0.8377
Data Shuffled
Epoch 185/15000 | train loss: 1.3166 | train_error 0.0341 | valid_error 0.8387
Data Shuffled
Epoch 186/15000 | train loss: 1.3129 | train_error 0.0339 | valid_error 0.8379
Data Shuffled
Epoch 187/15000 | train loss: 1.3170 | train_error 0.0348 | valid_error 0.8360
Data Shuffled
Epoch 188/15000 | train loss: 1.3127 | train_error 0.0339 | valid_error 0.8358
Data Shuffled
Epoch 189/15000 | train loss: 1.3150 | train_error 0.0334 | valid_error 0.8344
Data Shuffled
Epoch 190/15000 | train loss: 1.3204 | train_error 0.0334 | valid_erro

Epoch 270/15000 | train loss: 1.3971 | train_error 0.0301 | valid_error 0.7995
Data Shuffled
Epoch 271/15000 | train loss: 1.4029 | train_error 0.0294 | valid_error 0.7992
Data Shuffled
Epoch 272/15000 | train loss: 1.3968 | train_error 0.0344 | valid_error 0.8015
Data Shuffled
Epoch 273/15000 | train loss: 1.4015 | train_error 0.0300 | valid_error 0.8002
Data Shuffled
Epoch 274/15000 | train loss: 1.4007 | train_error 0.0337 | valid_error 0.7999
Data Shuffled
Epoch 275/15000 | train loss: 1.4098 | train_error 0.0297 | valid_error 0.7986
Data Shuffled
Epoch 276/15000 | train loss: 1.4055 | train_error 0.0296 | valid_error 0.8004
Data Shuffled
Epoch 277/15000 | train loss: 1.4044 | train_error 0.0302 | valid_error 0.7975
Data Shuffled
Epoch 278/15000 | train loss: 1.4076 | train_error 0.0308 | valid_error 0.7963
Data Shuffled
Epoch 279/15000 | train loss: 1.4131 | train_error 0.0316 | valid_error 0.7959
Data Shuffled
Epoch 280/15000 | train loss: 1.4132 | train_error 0.0304 | valid_erro

Epoch 360/15000 | train loss: 1.4892 | train_error 0.0270 | valid_error 0.7690
Data Shuffled
Epoch 361/15000 | train loss: 1.4888 | train_error 0.0269 | valid_error 0.7701
Data Shuffled
Epoch 362/15000 | train loss: 1.4889 | train_error 0.0277 | valid_error 0.7689
Data Shuffled
Epoch 363/15000 | train loss: 1.4891 | train_error 0.0298 | valid_error 0.7696
Data Shuffled
Epoch 364/15000 | train loss: 1.4890 | train_error 0.0278 | valid_error 0.7697
Data Shuffled
Epoch 365/15000 | train loss: 1.4886 | train_error 0.0275 | valid_error 0.7690
Data Shuffled
Epoch 366/15000 | train loss: 1.4923 | train_error 0.0272 | valid_error 0.7678
Data Shuffled
Epoch 367/15000 | train loss: 1.4876 | train_error 0.0278 | valid_error 0.7685
Data Shuffled
Epoch 368/15000 | train loss: 1.4926 | train_error 0.0266 | valid_error 0.7694
Data Shuffled
Epoch 369/15000 | train loss: 1.4908 | train_error 0.0276 | valid_error 0.7664
Data Shuffled
Epoch 370/15000 | train loss: 1.4953 | train_error 0.0264 | valid_erro

...

Epoch 97/500 | train loss: 1.2994 | train_error 0.0066 | valid_error 0.0094
Data Shuffled
Epoch 98/500 | train loss: 1.2893 | train_error 0.0063 | valid_error 0.0098
Data Shuffled
Epoch 99/500 | train loss: 1.2838 | train_error 0.0063 | valid_error 0.0094
Data Shuffled
Epoch 100/500 | train loss: 1.2780 | train_error 0.0063 | valid_error 0.0100
Data Shuffled
Epoch 101/500 | train loss: 1.2717 | train_error 0.0062 | valid_error 0.0098
Data Shuffled
Epoch 102/500 | train loss: 1.2650 | train_error 0.0062 | valid_error 0.0098
Data Shuffled
Epoch 103/500 | train loss: 1.2583 | train_error 0.0061 | valid_error 0.0094
Data Shuffled
Epoch 104/500 | train loss: 1.2524 | train_error 0.0062 | valid_error 0.0093
Data Shuffled
Epoch 105/500 | train loss: 1.2472 | train_error 0.0061 | valid_error 0.0097
Data Shuffled
Epoch 106/500 | train loss: 1.2404 | train_error 0.0060 | valid_error 0.0094
Data Shuffled
Epoch 107/500 | train loss: 1.2352 | train_error 0.0060 | valid_error 0.0095
Data Shuffled
Ep

Epoch 189/500 | train loss: 0.8723 | train_error 0.0041 | valid_error 0.0064
Data Shuffled
Epoch 190/500 | train loss: 0.8691 | train_error 0.0041 | valid_error 0.0064
Data Shuffled
Epoch 191/500 | train loss: 0.8670 | train_error 0.0042 | valid_error 0.0065
Data Shuffled
Epoch 192/500 | train loss: 0.8639 | train_error 0.0041 | valid_error 0.0065
Data Shuffled
Epoch 193/500 | train loss: 0.8601 | train_error 0.0042 | valid_error 0.0064
Data Shuffled
Epoch 194/500 | train loss: 0.8578 | train_error 0.0040 | valid_error 0.0061
Data Shuffled
Epoch 195/500 | train loss: 0.8546 | train_error 0.0043 | valid_error 0.0063
Data Shuffled
Epoch 196/500 | train loss: 0.8516 | train_error 0.0042 | valid_error 0.0063
Data Shuffled
Epoch 197/500 | train loss: 0.8494 | train_error 0.0042 | valid_error 0.0065
Data Shuffled
Epoch 198/500 | train loss: 0.8457 | train_error 0.0042 | valid_error 0.0061
Data Shuffled
Epoch 199/500 | train loss: 0.8433 | train_error 0.0040 | valid_error 0.0059
Data Shuffled

Epoch 281/500 | train loss: 0.7227 | train_error 0.0026 | valid_error 0.0036
Data Shuffled
Epoch 282/500 | train loss: 0.7209 | train_error 0.0027 | valid_error 0.0038
Data Shuffled
Epoch 283/500 | train loss: 0.7203 | train_error 0.0024 | valid_error 0.0035
Data Shuffled
Epoch 284/500 | train loss: 0.7186 | train_error 0.0025 | valid_error 0.0037
Data Shuffled
Epoch 285/500 | train loss: 0.7172 | train_error 0.0024 | valid_error 0.0035
Data Shuffled
Epoch 286/500 | train loss: 0.7154 | train_error 0.0024 | valid_error 0.0037
Data Shuffled
Epoch 287/500 | train loss: 0.7143 | train_error 0.0025 | valid_error 0.0036
Data Shuffled
Epoch 288/500 | train loss: 0.7132 | train_error 0.0023 | valid_error 0.0034
Data Shuffled
Epoch 289/500 | train loss: 0.7120 | train_error 0.0024 | valid_error 0.0034
Data Shuffled
Epoch 290/500 | train loss: 0.7110 | train_error 0.0023 | valid_error 0.0033
Data Shuffled
Epoch 291/500 | train loss: 0.7098 | train_error 0.0024 | valid_error 0.0036
Data Shuffled

Epoch 373/500 | train loss: 0.6112 | train_error 0.0016 | valid_error 0.0028
Data Shuffled
Epoch 374/500 | train loss: 0.6103 | train_error 0.0017 | valid_error 0.0028
Data Shuffled
Epoch 375/500 | train loss: 0.6094 | train_error 0.0017 | valid_error 0.0028
Data Shuffled
Epoch 376/500 | train loss: 0.6079 | train_error 0.0016 | valid_error 0.0029
Data Shuffled
Epoch 377/500 | train loss: 0.6068 | train_error 0.0016 | valid_error 0.0028
Data Shuffled
Epoch 378/500 | train loss: 0.6058 | train_error 0.0017 | valid_error 0.0028
Data Shuffled
Epoch 379/500 | train loss: 0.6050 | train_error 0.0016 | valid_error 0.0027
Data Shuffled
Epoch 380/500 | train loss: 0.6039 | train_error 0.0016 | valid_error 0.0028
Data Shuffled
Epoch 381/500 | train loss: 0.6028 | train_error 0.0015 | valid_error 0.0027
Data Shuffled
Epoch 382/500 | train loss: 0.6013 | train_error 0.0015 | valid_error 0.0029
Data Shuffled
Epoch 383/500 | train loss: 0.6005 | train_error 0.0015 | valid_error 0.0028
Data Shuffled

Epoch 465/500 | train loss: 0.5257 | train_error 0.0011 | valid_error 0.0027
Data Shuffled
Epoch 466/500 | train loss: 0.5252 | train_error 0.0010 | valid_error 0.0027
Data Shuffled
Epoch 467/500 | train loss: 0.5250 | train_error 0.0011 | valid_error 0.0025
Data Shuffled
Epoch 468/500 | train loss: 0.5240 | train_error 0.0011 | valid_error 0.0026
Data Shuffled
Epoch 469/500 | train loss: 0.5233 | train_error 0.0010 | valid_error 0.0024
Data Shuffled
Epoch 470/500 | train loss: 0.5225 | train_error 0.0010 | valid_error 0.0027
Data Shuffled
Epoch 471/500 | train loss: 0.5218 | train_error 0.0010 | valid_error 0.0025
Data Shuffled
Epoch 472/500 | train loss: 0.5211 | train_error 0.0010 | valid_error 0.0025
Data Shuffled
Epoch 473/500 | train loss: 0.5205 | train_error 0.0011 | valid_error 0.0024
Data Shuffled
Epoch 474/500 | train loss: 0.5197 | train_error 0.0010 | valid_error 0.0024
Data Shuffled
Epoch 475/500 | train loss: 0.5191 | train_error 0.0010 | valid_error 0.0024
Data Shuffled

In [38]:
ext_encoder = get_encoder(ext_train, ext_eval, sa_hidden_size=10)
save(ext_encoder, 'ext_encoder_better')

/data/anaconda/envs/fastai/lib/python3.6/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [144]:
ext_eval.shape

(2121, 24)

In [52]:
#test the sa
with torch.no_grad():
    res = torch.from_numpy(ext_eval.astype(np.float32))
    for encoder in ext_encoder:
        res, loss = ext_encoder[0].train()(res)
        print(abs(res.numpy() - ext_eval.astype(np.float32)).mean())
        print(loss)



0.41741797
tensor(3.5325)
0.58221215
tensor(3.6969)
0.68629974
tensor(3.8106)
0.7612964
tensor(3.8973)


## Creazione dei dataset

In [ ]:
# #parte utile per visualizzare i diversi indici da selezionare
# start_date = "2000-01-01"
# end_date = "2018-12-31"
# security = yfinance.Ticker('ASHR')# TODO trova mercato asiatico e indiano
# security_data = security.history(start=start_date, end=end_date, actions=False)
# # security_data = pd.DataFrame(security_data.values, index=security_data.index[::-1], columns=security_data.columns) # inv option
# opn.plot()

Consideriamo i seguenti indici:
-mercati sviluppati: S&P500(^GSPC), Dow Jones Industrial Average(^DJI)
-mercati nel mezzo: Hang Seng index in Hong Kong(^HSI), Nikkei 225 index in Tokyo(^N225)
-mercati in via di sviluppo: CSI 300 in mainland China (ASHR), Nifty 50 in India(^NSEI)

In [54]:
# indices = ['^GSPC', '^DJI', '^HSI', '^N225', 'ASHR','^NSEI']

In [57]:
indices = ['^GSPC']

In [ ]:
# feature_sets = ['open', 'ohlcv', 'ext','sa_ohlcv', 'sa_ext']

In [58]:
feature_sets = ['open', 'ohlcv', 'ext']

In [59]:
start_date="2000-01-01"
end_date="2018-12-31"

In [88]:
data.loc['2001-02-10':'2017-10-10'].copy()

Empty DataFrame
Columns: [Open, High, Low, Close, Volume, CloseL1, HighL1, LowL1, VolumeL1, EMA20, MA5, MA10, MA20, MACD, CCI, ATR, BollUp, BollDown, WVAD, MTM6, MTM12, SMI, ROC, USDOpen]
Index: []

[0 rows x 24 columns]

In [92]:
max(data.index[0], data.index[1]) 

Timestamp('2000-01-12 00:00:00')

In [94]:
# i dataset sono organizzati nel seguente modo
# -indice
#  -dati originali : dataframe pandas
#  -features: dizionario di featureset
start_date="2000-01-01"
end_date="2018-12-31"
start_data = None
end_data = None
datasets = {}
for index in indices:
    datasets[index] = {}
    datasets[index]["original"] = get_index(index=index, start_date=start_date, end_date=end_date)
    datasets[index]["features"] = {}
    datasets[index]["target"] = None
    for feature_set in feature_sets:
        data = get_dataset_by_name(datasets[index]["original"], name=feature_set)
        data.dropna(inplace=True)
        if(start_data is None):
            start_data = data.index.min()
        else:
            start_data = max(data.index.min(), start_data)
        if(end_data is None):
            end_data = data.index.max()
        else:
            end_data = min(data.index.max(), end_data)
        datasets[index]["features"][feature_set] = data.drop("Close", axis=1)
        if(datasets[index]["target"] is None):
            datasets[index]["target"] = data["Close"]
            
for index in datasets:      
    datasets[index]["target"] = datasets[index]["target"].iloc[start_data:end_data] 
    for feature_set in datasets[index]["features"]:  
        feature_set = feature_set.loc[start_data:end_data].copy()

TypeError: cannot do slice indexing on <class 'pandas.core.indexes.datetimes.DatetimeIndex'> with these indexers [2001-01-03 00:00:00] of <class 'pandas._libs.tslibs.timestamps.Timestamp'>

## Training dei modelli

In [ ]:
#dataset è della forma (X,Y)
def set_dataset(data_set):
    global X
    global Y
    global X_train
    global Y_train
    global X_val
    global Y_val
    global X_train_f
    global Y_train_f
    global X_val_f
    global Y_val_f
    global X_f
    global Y_f
    global half
    global half_split
    X= data_set[0]
    Y= data_set[1]
    X_train, X_val, Y_train, Y_val = split_before_2010_06_30(X, Y)
    X_f = X.to_numpy().astype(np.float32)
    Y_f = Y.to_numpy().astype(np.float32)[...,None]
    X_train_f = X_train.to_numpy().astype(np.float32)
    Y_train_f = Y_train.to_numpy().astype(np.float32)[...,None]
    X_val_f = X_val.to_numpy().astype(np.float32)
    Y_val_f = Y_val.to_numpy().astype(np.float32)[...,None] 
    l1 = len(np.split(X_f, [len(X_f)//2])[0])
    l2 = len(np.split(X_f, [len(X_f)//2])[1])
    half = PredefinedSplit(np.concatenate((np.ones(l1)*-1,np.ones(l2))))
    half_split =  CVSplit(cv=half, stratified=False, random_state=None)

In [ ]:
# ohlcv_encoder = get_encoder(split_before_2010_06_30(data_set["ohlcv"][0], data_set["ohlcv"][1])[0])
# ohlcv_encoder = get_encoder(data_set["ohlcv"][0])

In [100]:
tss = TimeSeriesSplit(3)
tss_split = CVSplit(cv=tss, stratified=False, random_state=None)

In [ ]:
models = {}

In [ ]:
# # modello del paper di Moro
# #SALVATO!
# OK
n_days= 10
lstm_moro = NeuralNetRegressor(
    module=SequenceDouble,
    optimizer=optim.SGD,
    batch_size=n_days,
    criterion=RNNMSELoss,
    max_epochs=200, # usato nel paper
    train_split=None,
    
    module__nb_features=X_f.shape[1],
    module__hidden_size=256,
    optimizer__lr=0.01,
    optimizer__weight_decay=0,
    optimizer__momentum=0.9
)
models['lstm_moro'] = (lstm_moro, 'open')

In [ ]:
#modello del paper che usa stacked autoencoders usando LSTM paper Moro
# TODO trova epoche ottimali
n_days = 10

lstm_sa = NeuralNetRegressor(
    module=SequenceDouble,
    optimizer=optim.SGD,
    batch_size=n_days,
    criterion=RNNMSELoss,
    max_epochs=5000, #TODO
    train_split=None,
    
    module__nb_features=X_f.shape[1],
    module__hidden_size=256,
    optimizer__lr=0.01,
    optimizer__weight_decay=0,
    optimizer__momentum=0.9
)

models['lstm_sa'] = (lstm_sa,'sa_ext')

In [ ]:
# models = {'lstm_moro':(lstm_moro, 'open'), 'lstm_sa':(lstm_sa_d_1000,'sa_ext'), 'lstm_att_ohlcv':(lstm_att_d_600,'ohlcv'), 'lstm_att_ext':(lstm_att_d_600,'ext'), 'lstm_sa_att,':(lstm_sa_att_1500, 'sa_ext')}

In [ ]:
# TODO assicurarsi che tutti i feature set abbiano la stessa luinghezza

In [ ]:
scores = pd.DataFrame()#TODO define structure salva risultati ad ogni iterazione in csv
for index_data in indexes_data:
    for i, (train, val) in enumerate(tss.split(index_data["target"]), start=1):
        
        feature_sets = {}
        
#         train_dates = index_data["original"].index[train]
#         val_dates = index_data["original"].index[val]
        
        y = index_data["target"]
        opn_dataset = index_data["features"]['open']
        ohlcv_dataset = index_data["features"]['ohlcv']
        ext_dataset = index_data["features"]['ext']

        #data split
        
        opn_train, opn_val = opn_dataset.iloc[train_dates].to_numpy(np.float32), opn_dataset.iloc[val_dates].to_numpy(np.float32)
        ohlcv_train, ohlcv_val = ohlcv_dataset.iloc[train_dates].to_numpy(np.float32), ohlcv_dataset.iloc[val_dates].to_numpy(np.float32)
        ext_train, ext_val = ext_dataset.iloc[train_dates].to_numpy(np.float32), ext_dataset.iloc[val_dates].to_numpy(np.float32)
        y_train, y_val = y.iloc[train_dates].to_numpy(np.float32)[...,None], y.iloc[val_dates].to_numpy(np.float32)[...,None]
        
        #data scale
        opn_scaler = StandardScaler()
        ohlcv_scaler = StandardScaler()
        ext_scaler = StandardScaler()
        y_scaler = StandardScaler()

        opn_train = ohlcv_scaler.fit_transform(opn_train)
        ohlcv_train = ohlcv_scaler.fit_transform(ohlcv_train)
        ext_train = ext_scaler.fit_transform(ext_train)
        y_train = y_scaler.fit_transform(y_train)
        opn_val = ohlcv_scaler.transform(opn_val)
        ohlcv_val = ohlcv_scaler.ransform(ohlcv_val)
        ext_val = ext_scaler.transform(ext_val)
        y_val = y_scaler.transform(y_val)
        
        ext_encoder = get_encoder(ext_train, ext_val, sa_hidden_size=10)
        ext_sa_train, ext_sa_val = encode(ext_train, ext_encoder), encode(ext_val, ext_encoder)
        
        feature_sets['open'] = (opn_train, opn_val, opn_scaler)
        feature_sets['ohlcv'] = (ohlcv_train, ohlcv_val, ohlcv_scaler)
        feature_sets['ext'] = (ext_train, ext_val, ext_scaler)
        feature_sets['ext_sa'] = (ext_sa_train, ext_sa_val, ext_scaler)
        
        #training
        for key, info in models:
            model = info[0]
            x_train, x_val, scaler = feature_sets[info[1]]
            #TODO train with no validation set
            model.fit(x_train, y_train)
            #TODO save the model
            # TODO evalutate the model and add its scores to a dataframe
            for i in range(10, len(y_val)): 
                pred = model.predict(x_val[i-10:i].to_numpy().astype(np.float32))[-1] #in input vengono dati 10 gionri
                scaler.
        
        print("FOLD {}".format(i))
        train_dates = X.index[train]
        val_dates = X.index[val]
        print("Training set da {} a {}".format(train_dates.min(), train_dates.max()))
        print("Validation set da {} a {}".format(val_dates.min(), val_dates.max()))

        
        
    

## Metriche

In [125]:
## Calcola la loss solo sull'utlimo elemento del batch
class RNNMSELoss(MSELoss):
    def __call__(self, input, target):
        return super().__call__(input, target[-1])

## Modelli

In [ ]:
# set_dataset('sa_ohlcv')

In [ ]:
# # NON CONVERGE!!
# #modello del paper che usa stacked autoencoders usando LSTM originale TODO test
# n_days = 4 #nel paper chiamati delays 

# lstm_sa = NeuralNetRegressor(
#     module=Sequence,
#     optimizer=optim.Adam,
#     batch_size=n_days,
#     criterion=RNNMSELoss,
#     max_epochs=5000,
#     train_split=half_split,
    
#     module__nb_features=X.shape[1],
#     module__hidden_size=1,
#     module__nb_layers=5,
#     optimizer__lr=0.01
# )

In [ ]:
# lstm_sa.fit(X_f, Y_f)

In [ ]:
# set_dataset('sa_ext')

In [120]:
train_dates, val_dates = list(tss.split(datasets['^GSPC']["target"]))[0]

In [140]:
y = datasets['^GSPC']["target"]
ext_dataset = datasets['^GSPC']["features"]['ext']
ext_scaler = StandardScaler()
y_scaler = StandardScaler()
# sa = load('ext_encoder_better')
ext_train, ext_val = ext_scaler.fit_transform(ext_dataset.iloc[train_dates].to_numpy(np.float32)), ext_scaler.transform(ext_dataset.iloc[val_dates].to_numpy(np.float32))
y_train, y_val = y_scaler.fit_transform(y.iloc[train_dates].to_numpy(np.float32)[...,None]), y_scaler.transform(y.iloc[val_dates].to_numpy(np.float32)[...,None])

In [145]:
ext_train.shape

(1193, 23)

In [149]:
ext_encoder = get_encoder(ext_train, ext_val, sa_hidden_size=10)
save(ext_encoder, 'ext_encoder_better')

Data Shuffled
Epoch 1/15000 | train loss: 5.7844 | train_error 0.7315 | valid_error 1.9610
Data Shuffled
Epoch 2/15000 | train loss: 7.6817 | train_error 0.6080 | valid_error 1.7961
Data Shuffled
Epoch 3/15000 | train loss: 8.2292 | train_error 0.4844 | valid_error 1.6577
Data Shuffled
Epoch 4/15000 | train loss: 7.6877 | train_error 0.4234 | valid_error 1.5608
Data Shuffled
Epoch 5/15000 | train loss: 7.1450 | train_error 0.3899 | valid_error 1.5301
Data Shuffled
Epoch 6/15000 | train loss: 6.7725 | train_error 0.3636 | valid_error 1.5200
Data Shuffled
Epoch 7/15000 | train loss: 6.6422 | train_error 0.3432 | valid_error 1.4993
Data Shuffled
Epoch 8/15000 | train loss: 6.5075 | train_error 0.3262 | valid_error 1.4891
Data Shuffled
Epoch 9/15000 | train loss: 6.5762 | train_error 0.3118 | valid_error 1.4693
Data Shuffled
Epoch 10/15000 | train loss: 6.5859 | train_error 0.2997 | valid_error 1.4635
Data Shuffled
Epoch 11/15000 | train loss: 6.6595 | train_error 0.2886 | valid_error 1.45

Epoch 91/15000 | train loss: 7.5482 | train_error 0.1059 | valid_error 1.2197
Data Shuffled
Epoch 92/15000 | train loss: 7.5806 | train_error 0.1040 | valid_error 1.2059
Data Shuffled
Epoch 93/15000 | train loss: 7.4696 | train_error 0.1034 | valid_error 1.2081
Data Shuffled
Epoch 94/15000 | train loss: 7.5049 | train_error 0.1040 | valid_error 1.2109
Data Shuffled
Epoch 95/15000 | train loss: 7.4771 | train_error 0.1019 | valid_error 1.1971
Data Shuffled
Epoch 96/15000 | train loss: 7.4976 | train_error 0.1007 | valid_error 1.1994
Data Shuffled
Epoch 97/15000 | train loss: 7.4728 | train_error 0.0995 | valid_error 1.1962
Data Shuffled
Epoch 98/15000 | train loss: 7.4005 | train_error 0.0992 | valid_error 1.1936
Data Shuffled
Epoch 99/15000 | train loss: 7.3823 | train_error 0.0991 | valid_error 1.1999
Data Shuffled
Epoch 100/15000 | train loss: 7.3185 | train_error 0.0982 | valid_error 1.1822
Data Shuffled
Epoch 101/15000 | train loss: 7.2720 | train_error 0.0968 | valid_error 1.1898


Epoch 181/15000 | train loss: 6.6833 | train_error 0.0625 | valid_error 1.0364
Data Shuffled
Epoch 182/15000 | train loss: 6.7170 | train_error 0.0619 | valid_error 1.0396
Data Shuffled
Epoch 183/15000 | train loss: 6.7030 | train_error 0.0621 | valid_error 1.0417
Data Shuffled
Epoch 184/15000 | train loss: 6.6910 | train_error 0.0619 | valid_error 1.0419
Data Shuffled
Epoch 185/15000 | train loss: 6.7258 | train_error 0.0621 | valid_error 1.0414
Data Shuffled
Epoch 186/15000 | train loss: 6.6755 | train_error 0.0606 | valid_error 1.0309
Data Shuffled
Epoch 187/15000 | train loss: 6.6954 | train_error 0.0605 | valid_error 1.0313
Data Shuffled
Epoch 188/15000 | train loss: 6.7020 | train_error 0.0601 | valid_error 1.0274
Data Shuffled
Epoch 189/15000 | train loss: 6.6899 | train_error 0.0601 | valid_error 1.0304
Data Shuffled
Epoch 190/15000 | train loss: 6.6971 | train_error 0.0608 | valid_error 1.0380
Data Shuffled
Epoch 191/15000 | train loss: 6.7050 | train_error 0.0594 | valid_erro

Epoch 272/15000 | train loss: 6.4371 | train_error 0.0452 | valid_error 0.9475
Data Shuffled
Epoch 273/15000 | train loss: 6.4794 | train_error 0.0447 | valid_error 0.9416
Data Shuffled
Epoch 274/15000 | train loss: 6.4787 | train_error 0.0448 | valid_error 0.9436
Data Shuffled
Epoch 275/15000 | train loss: 6.4542 | train_error 0.0451 | valid_error 0.9361
Data Shuffled
Epoch 276/15000 | train loss: 6.4506 | train_error 0.0452 | valid_error 0.9455
Data Shuffled
Epoch 277/15000 | train loss: 6.4514 | train_error 0.0446 | valid_error 0.9354
Data Shuffled
Epoch 278/15000 | train loss: 6.4500 | train_error 0.0449 | valid_error 0.9357
Data Shuffled
Epoch 279/15000 | train loss: 6.4521 | train_error 0.0445 | valid_error 0.9421
Data Shuffled
Epoch 280/15000 | train loss: 6.4620 | train_error 0.0445 | valid_error 0.9390
Data Shuffled
Epoch 281/15000 | train loss: 6.4598 | train_error 0.0441 | valid_error 0.9354
Data Shuffled
Epoch 282/15000 | train loss: 6.4447 | train_error 0.0444 | valid_erro

Epoch 364/15000 | train loss: 6.4094 | train_error 0.0394 | valid_error 0.8828
Data Shuffled
Epoch 365/15000 | train loss: 6.3891 | train_error 0.0397 | valid_error 0.8865
Data Shuffled
Epoch 366/15000 | train loss: 6.3808 | train_error 0.0398 | valid_error 0.8856
Data Shuffled
Epoch 367/15000 | train loss: 6.4086 | train_error 0.0394 | valid_error 0.8811
Data Shuffled
Epoch 368/15000 | train loss: 6.4228 | train_error 0.0403 | valid_error 0.8748
Data Shuffled
Epoch 369/15000 | train loss: 6.4107 | train_error 0.0395 | valid_error 0.8773
Data Shuffled
Epoch 370/15000 | train loss: 6.4177 | train_error 0.0397 | valid_error 0.8755
Data Shuffled
Epoch 371/15000 | train loss: 6.4221 | train_error 0.0400 | valid_error 0.8773
Data Shuffled
Epoch 372/15000 | train loss: 6.4011 | train_error 0.0395 | valid_error 0.8790
Data Shuffled
Epoch 373/15000 | train loss: 6.4132 | train_error 0.0392 | valid_error 0.8788
Data Shuffled
Epoch 374/15000 | train loss: 6.4303 | train_error 0.0388 | valid_erro

...

Epoch 1540/15000 | train loss: 5.9865 | train_error 0.0260 | valid_error 0.6507
Data Shuffled
Epoch 1541/15000 | train loss: 5.9697 | train_error 0.0282 | valid_error 0.6550
Data Shuffled
Epoch 1542/15000 | train loss: 5.9724 | train_error 0.0268 | valid_error 0.6537
Data Shuffled
Epoch 1543/15000 | train loss: 5.9928 | train_error 0.0262 | valid_error 0.6508
Data Shuffled
Epoch 1544/15000 | train loss: 5.9826 | train_error 0.0249 | valid_error 0.6437
Data Shuffled
Epoch 1545/15000 | train loss: 5.9645 | train_error 0.0254 | valid_error 0.6403
Data Shuffled
Epoch 1546/15000 | train loss: 5.9802 | train_error 0.0258 | valid_error 0.6398
Data Shuffled
Epoch 1547/15000 | train loss: 5.9857 | train_error 0.0262 | valid_error 0.6378
Data Shuffled
Epoch 1548/15000 | train loss: 5.9786 | train_error 0.0251 | valid_error 0.6456
Data Shuffled
Epoch 1549/15000 | train loss: 5.9539 | train_error 0.0270 | valid_error 0.6552
Data Shuffled
Epoch 1550/15000 | train loss: 5.9582 | train_error 0.0255 |

Epoch 1628/15000 | train loss: 5.9175 | train_error 0.0246 | valid_error 0.6409
Data Shuffled
Epoch 1629/15000 | train loss: 5.9255 | train_error 0.0246 | valid_error 0.6422
Data Shuffled
Epoch 1630/15000 | train loss: 5.9069 | train_error 0.0256 | valid_error 0.6307
Data Shuffled
Epoch 1631/15000 | train loss: 5.9368 | train_error 0.0246 | valid_error 0.6353
Data Shuffled
Epoch 1632/15000 | train loss: 5.9181 | train_error 0.0255 | valid_error 0.6310
Data Shuffled
Epoch 1633/15000 | train loss: 5.9287 | train_error 0.0254 | valid_error 0.6441
Data Shuffled
Epoch 1634/15000 | train loss: 5.9152 | train_error 0.0245 | valid_error 0.6363
Data Shuffled
Epoch 1635/15000 | train loss: 5.9130 | train_error 0.0254 | valid_error 0.6428
Data Shuffled
Epoch 1636/15000 | train loss: 5.9225 | train_error 0.0255 | valid_error 0.6452
Data Shuffled
Epoch 1637/15000 | train loss: 5.9116 | train_error 0.0254 | valid_error 0.6444
Data Shuffled
Epoch 1638/15000 | train loss: 5.9198 | train_error 0.0247 |

Epoch 1718/15000 | train loss: 5.8883 | train_error 0.0248 | valid_error 0.6292
Data Shuffled
Epoch 1719/15000 | train loss: 5.8794 | train_error 0.0240 | valid_error 0.6370
Data Shuffled
Epoch 1720/15000 | train loss: 5.8679 | train_error 0.0236 | valid_error 0.6324
Data Shuffled
Epoch 1721/15000 | train loss: 5.8629 | train_error 0.0238 | valid_error 0.6371
Data Shuffled
Epoch 1722/15000 | train loss: 5.8770 | train_error 0.0236 | valid_error 0.6344
Data Shuffled
Epoch 1723/15000 | train loss: 5.8757 | train_error 0.0237 | valid_error 0.6384
Data Shuffled
Epoch 1724/15000 | train loss: 5.8535 | train_error 0.0236 | valid_error 0.6363
Data Shuffled
Epoch 1725/15000 | train loss: 5.8750 | train_error 0.0236 | valid_error 0.6332
Data Shuffled
Epoch 1726/15000 | train loss: 5.8518 | train_error 0.0234 | valid_error 0.6327
Data Shuffled
Epoch 1727/15000 | train loss: 5.8667 | train_error 0.0242 | valid_error 0.6275
Data Shuffled
Epoch 1728/15000 | train loss: 5.8824 | train_error 0.0242 |

Epoch 1809/15000 | train loss: 5.8444 | train_error 0.0233 | valid_error 0.6311
Data Shuffled
Epoch 1810/15000 | train loss: 5.8460 | train_error 0.0242 | valid_error 0.6214
Data Shuffled
Epoch 1811/15000 | train loss: 5.8495 | train_error 0.0239 | valid_error 0.6230
Data Shuffled
Epoch 1812/15000 | train loss: 5.8624 | train_error 0.0236 | valid_error 0.6264
Data Shuffled
Epoch 1813/15000 | train loss: 5.8730 | train_error 0.0231 | valid_error 0.6278
Data Shuffled
Epoch 1814/15000 | train loss: 5.8505 | train_error 0.0231 | valid_error 0.6259
Data Shuffled
Epoch 1815/15000 | train loss: 5.8401 | train_error 0.0233 | valid_error 0.6295
Data Shuffled
Epoch 1816/15000 | train loss: 5.8579 | train_error 0.0228 | valid_error 0.6310
Data Shuffled
Epoch 1817/15000 | train loss: 5.8640 | train_error 0.0238 | valid_error 0.6295
Data Shuffled
Epoch 1818/15000 | train loss: 5.8613 | train_error 0.0238 | valid_error 0.6289
Data Shuffled
Epoch 1819/15000 | train loss: 5.8736 | train_error 0.0242 |

KeyboardInterrupt: 

In [ ]:
ext_sa_train, ext_sa_val = encode(ext_train, ext_encoder), encode(ext_val, ext_encoder)

In [132]:
ext_train.shape[1]

23

In [133]:
#modello del paper che usa stacked autoencoders usando LSTM paper Moro
n_days = 10

#TODO test e trova numero epoche

lstm_sa = NeuralNetRegressor(
    module=SequenceDouble,#TODO usa delays come nel paper, fallo in altro modello, questo usalo per paragone
    optimizer=optim.SGD,
    batch_size=n_days,
    criterion=RNNMSELoss,
    max_epochs=5000, # trovato empiricamente 1050, da portare oltre(5000 nel paper)
    train_split=None,
    callbacks=[callbacks.Checkpoint(monitor='train_loss_best', f_pickle='lstm_sa_best')],
    
    module__nb_features=ext_train.shape[1],
    module__hidden_size=256,
    optimizer__lr=0.01,
    optimizer__weight_decay=0,
    optimizer__momentum=0.9
)

In [135]:
lstm_sa

<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=SequenceDouble(
    (lstm1): LSTM(23, 256)
    (lstm2): LSTM(256, 512)
    (lin): Linear(in_features=512, out_features=1, bias=True)
  ),
)

In [134]:
lstm_sa.fit(ext_train, y_train)

/data/anaconda/envs/fastai/lib/python3.6/site-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([23])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


  epoch    train_loss    cp     dur
-------  ------------  ----  ------
      1           nan        3.2066
      2           nan        3.5067
      3           nan        2.8704


<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=SequenceDouble(
    (lstm1): LSTM(23, 256)
    (lstm2): LSTM(256, 512)
    (lin): Linear(in_features=512, out_features=1, bias=True)
  ),
)

In [ ]:
# save(lstm_sa_d, 'lstm_sa_d_1000')

In [ ]:
# set_dataset('ohlcv')

In [ ]:
ohlcv_scaler = StandardScaler()

In [151]:
y = datasets['^GSPC']["target"]
ext_dataset = datasets['^GSPC']["features"]['ohlcv']
ext_scaler = StandardScaler()
y_scaler = StandardScaler()
# sa = load('ext_encoder_better')
ext_train, ext_val = ext_scaler.fit_transform(ext_dataset.iloc[train_dates].to_numpy(np.float32)), ext_scaler.transform(ext_dataset.iloc[val_dates].to_numpy(np.float32))
y_train, y_val = y_scaler.fit_transform(y.iloc[train_dates].to_numpy(np.float32)[...,None]), y_scaler.transform(y.iloc[val_dates].to_numpy(np.float32)[...,None])

In [152]:
ext_train.shape

(1193, 4)

In [166]:
#modello del paper che usa attention mechanism
#TODO testa se funziona e trova epoche
n_days = 10

lstm_att = NeuralNetRegressor(
    module=SequenceDoubleAtt,
    optimizer=optim.Adam,
    batch_size=n_days,
    criterion=RNNMSELoss,
    max_epochs=600, # trovato 280
    train_split=None,
    callbacks=[callbacks.Checkpoint(monitor='train_loss_best', f_pickle='lstm_att_best')],
    
    module__nb_features=ext_train.shape[1],
    module__hidden_size=256,
    optimizer__lr=0.0001,
#     optimizer__weight_decay=0,
#     optimizer__momentum=0.9
)

In [167]:
lstm_att.fit(ext_train, y_train) #TODO fai funzionare

TypeError: super(type, obj): obj must be an instance or subtype of type

In [ ]:
save(lstm_att_d, 'lstm_att_d_600')

In [ ]:
set_dataset('sa_ext)

In [ ]:
#modello del paper che usa attention mechanism
#TODO trova numero epoche
n_days = 10

lstm_sa_att = NeuralNetRegressor(
    module=SequenceDoubleAtt,
    optimizer=optim.Adam,
    batch_size=n_days,
    criterion=RNNMSELoss,
    max_epochs=5000,
    train_split=half_split,
    callbacks=[callbacks.Checkpoint(monitor='valid_loss_best', f_pickle='lstm_sa_att_best')],
    
    module__nb_features=X_f.shape[1],
    module__hidden_size=256,
    optimizer__lr=0.0001,
#     optimizer__weight_decay=0,
#     optimizer__momentum=0.9
)

In [ ]:
# lstm_sa_att_d.fit(X_f, Y_f)

In [ ]:
# save(lstm_sa_att_d, 'lstm_sa_att_sa_1500')

## Valutazione del training

In [ ]:
# lstm_test = lstm_sa_att_d

In [ ]:
# lstm_test.history[10].keys()

In [ ]:
# lstm = lstm_test
# valid_losses = lstm.history[:, 'valid_loss']
# train_losses = lstm.history[:, 'train_loss']
# plt.figure(figsize=(12,7))
# plt.plot(valid_losses, label='valid_loss')
# plt.plot(train_losses, label='train_loss')
# # plt.xticks(np.arange(len(valid_losses)+1, step=50))
# plt.legend()

# Test
Dopo aver trovato i parametri migliori i modelli vengono testati facendo training sulla metà dei dati e predizioni sull'altra metà

In [ ]:
datasets.keys()

In [ ]:
# models = ['lstm_sa_d', 'lstm_moro', 'lstm_sa_d_1000', 'lstm_att_d_600', 'lstm_sa_att_sa_1500']

In [ ]:
models = [('lstm_moro', 'open'), ('lstm_sa_d_1000','sa_ohlcv'), ('lstm_att_d_600','ohlcv'), ('lstm_sa_att_1500', 'ohlcv')]

In [ ]:
model = models[0]

In [ ]:
lstm_test = load(model[0])

In [ ]:
datasets["^GSPC"]["features"].keys()

In [ ]:
market = "^DJI"
feature_set = model[1]

In [ ]:
datasets[market]["original"]["Open"].plot()

In [ ]:
set_dataset(datasets[market]["features"][feature_set])
opn = datasets[market]["original"]["Open"]
close = datasets[market]["original"]["Close"]

In [ ]:
# i primi 51 giorni non vengonon considerati poichè servono per il calcolo dell'inversa dello z_index
Y_original = close.copy()
Y_preds = Y_original.copy()

for i in range(51, len(Y_val)): 
    pred = lstm_test.predict(X_val[i-10:i].to_numpy().astype(np.float32))[0] #in input vengono dati 10 gionri
    #denormalization
    previous_serie = Y_original[:i]
    Y_preds[i] = z_score_inv(previous_serie[-50:], pred) #il z score viene calcolato su 50 gionri
Y_preds = Y_preds[51:]
Y_original = Y_original.reindex_like(Y_preds, copy=False)

In [ ]:
plt.figure(figsize=(15,7))
Y_preds.plot(label='close_pred')
Y_original.plot(label='close_true')
plt.legend()

In [ ]:
Y_original.max()

In [ ]:
sklearn.metrics.mean_squared_error(Y_original, Y_preds)

In [ ]:
score = pd.Series(index=["MAPE","RMSPE", "R2", "ROI", "ROI Ideal", "ROI vs ideal"], dtype=np.float32)
score["MAPE"] = sklearn.metrics.mean_absolute_error(Y_original, Y_preds)/Y_preds.mean()
score["RMSPE"] = np.sqrt(sklearn.metrics.mean_squared_error(Y_original, Y_preds))/Y_original.mean()
score["R2"] = sklearn.metrics.r2_score(Y_original, Y_preds)
score["ROI"] = roi(Y_original,Y_preds, opn)
score["ROI Ideal"] = roi(Y_original,Y_original, opn)
score["ROI vs ideal"] = abs(roi(Y_original,Y_preds, opn) - roi(Y_original,Y_original, opn))

In [ ]:
score

In [ ]:
scores = pd.DataFrame(index=score.index)


In [ ]:
show_transaction = False
initial_capital = 10000
capital = initial_capital
cap_history = Y_preds.copy()
holding=False
opns = opn.reindex_like(Y_preds).copy()
buy_history = opn.reindex_like(Y_preds).copy()
sell_history = opn.reindex_like(Y_preds).copy()
for date in Y_preds.index:
    buy_history[date] = None
    sell_history[date] = None
    pred = Y_preds[date]
    true = Y_original[date]
    buy = Y_preds[date] - opns[date] > 0
    delta_true = Y_original[date] - opns[date]
    if(buy and not holding): # buy
        if(show_transaction):
            capital -= opns[date]
        holding = True
        buy_history[date] = capital
    elif(not buy and holding): #sell
        if(show_transaction):
            capital += opns[date]
        holding = False
        sell_history[date] = capital
    if(holding):
        capital+= delta_true
    cap_history[date] = capital
if(holding and show_transaction):
    capital += opns[-1]
    holding = False
abs_gain = capital - initial_capital
perc_gain = (abs_gain) / initial_capital
print(f"guadagno assoluto: {abs_gain}")
print(f"guadagno percentuale: {perc_gain}")

In [ ]:
plt.figure(figsize=(15,7))
plt.scatter(buy_history.index, buy_history, c='green', label='buy', linewidths=0.001)
plt.scatter(sell_history.index, sell_history, c= 'red', label='sell', linewidths=0.001)
cap_history.plot(label='capital')
plt.legend()